In [ ]:
import sys
import os
import importlib as implib

import re

from collections import OrderedDict as odict

import numpy as np
import pandas as pd

from tqdm import tqdm

In [ ]:
import autods as ads

# Tests unitaires et d'intégration module *autods*.

## 1. Classe DataSet

In [ ]:
dfData = pd.DataFrame(columns=['Date', 'TrucDec', 'Espece', 'Point', 'Effort', 'Distance'],
                      data=[('2019-05-13', 3.5, 'TURMER', 23, 2,   83),
                            ('2019-05-15', np.nan, 'TURMER', 23, 2,   27.355),
                            ('2019-05-13', 0, 'ALAARV', 29, 2,   56.85),
                            ('2019-04-03', 1.325, 'PRUMOD', 53, 1.3,  7.2),
                            ('2019-06-01', 2, 'PHICOL', 12, 1,  np.nan),
                            ('2019-06-19', np.nan, 'PHICOL', 17, 0.5, np.nan),
                           ])
dfData['Region'] = 'ACDC'
dfData['Surface'] = '2400'
dfData

In [ ]:
ds = ads.DataSet(dfData, decimalFields=['Effort', 'Distance', 'TrucDec'])

## 2. Classes XXEngine

### a. Instanciation et détection de Distance

In [ ]:
_ = implib.reload(ads)

In [ ]:
try:
    eng = ads.MCDSEngine(workDir=os.path.join('AutoDS', 'test out'))
    print('Error: Should have raised an AssertionError !')
except AssertionError as exc:
    print('Good forbidden chars detection:', exc)

In [ ]:
eng = ads.MCDSEngine(workDir=os.path.join('AutoDS', 'mcds-out'))

In [ ]:
_ = eng.setupRunFolder(runPrefix='uni') # Unit tests

### b. Génération fichier de données en entrée de MCDS

In [ ]:
dataFileName = eng.buildDataFile(dataSet=ds)

### c. Génération fichier de "commandes"

In [ ]:
cmdFileName = eng.buildCmdFile(estimKeyFn='HNORMAL', estimAdjustFn='COSINE',
                               estimCriterion='AIC', cvInterval=95)

### d. Execution en mode "debug"

(génération des fichiers cmd et data, mais pas d'appel à l'exécutable)

In [ ]:
runCode, runDir = eng.run(ds, realRun=False, runPrefix='int',
                          estimKeyFn='UNIFORM', estimAdjustFn='POLY',
                          estimCriterion='AIC', cvInterval=95)
assert runCode == 0, 'Should have NOT run (run code = 0)'
runDir

### e. Exécution réelle

In [ ]:
runCode, runDir = eng.run(ds, realRun=True, runPrefix='int',
                          estimKeyFn='UNIFORM', estimAdjustFn='POLY',
                          estimCriterion='AIC', cvInterval=95)
assert runCode == 2, 'Should have run with warnings (run code = 2)'
runDir

### f. Génération fichier de données en entrée pour Distance

(mode 'point transect' uniquement pour le moment)

In [ ]:
os.makedirs(os.path.join(eng.workDir, 'distance-in'), exist_ok=True)

In [ ]:
distDataFileName = \
    eng.buildDistanceDataFile(ds, tgtFilePathName=os.path.join(eng.workDir, 'distance-in', 'import-data-noextra.txt'))

In [ ]:
distDataFileName = \
    eng.buildDistanceDataFile(ds, tgtFilePathName=os.path.join(eng.workDir, 'distance-in', 'import-data-withextra.txt'),
                              withExtraFields=True)

### g. classe ResultsSet

In [ ]:
_ = implib.reload(ads)

In [ ]:
customCols = pd.MultiIndex.from_tuples([('id', 'index', 'Value'),
                                        ('sample', 'species', 'Value'),
                                        ('sample', 'periods', 'Value'),
                                        ('sample', 'duration', 'Value'),
                                        ('variant', 'precision', 'Value')])

rs = ads.ResultsSet(analysisClass=ads.MCDSAnalysis, customColumns=customCols)

In [ ]:
assert rs.dfData.empty

In [ ]:
rs.dfData

In [ ]:
ads.MCDSAnalysis.EngineClass.statModColumns()

# Tests de validation module autods

## 1. MCDSEngine : Génération de fichiers d'entrée pour Distance

* via un jeu de fichiers d'entrée bruts Excel, et leur export de référence, éprouvé dans Distance,
* et comparaison du produit de XXEngine.buildDistanceDataFile à cette référence.

In [ ]:
class TestCase(object):
    pass
class TCDistance(TestCase):
    def __init__(self, inFileName, decimalFields, withExtraFields, refOutFileName):
        self.inFileName, self.decimalFields, self.withExtraFields, self.refOutFileName = \
            inFileName, decimalFields, withExtraFields, refOutFileName

In [ ]:
testCases = [TCDistance(inFileName='ALAARV-saisie-ttes-cols.xlsx', decimalFields=['EFFORT', 'DISTANCE', 'NOMBRE'],
                        refOutFileName='ALAARV-saisie-5-cols.txt', withExtraFields=False),
             TCDistance(inFileName='ALAARV-saisie-ttes-cols.xlsx', decimalFields=['EFFORT', 'DISTANCE', 'NOMBRE'],
                        refOutFileName='ALAARV-saisie-ttes-cols.txt', withExtraFields=True)]

In [ ]:
eng = ads.MCDSEngine(workDir=os.path.join('AutoDS', 'mcds-out'))

In [ ]:
fails = 0
for ind, tc in enumerate(testCases):
    
    print('#', ind, ':', tc.inFileName)

    # Create data set
    ds = ads.DataSet(dfData=pd.read_excel(os.path.join('AutoDS', 'refin', tc.inFileName)),
                     decimalFields=tc.decimalFields)
    
    # Build distance import data file
    ofn = os.path.join(eng.workDir, 'distance-in', tc.refOutFileName)
    ofn = eng.buildDistanceDataFile(dataSet=ds, tgtFilePathName=ofn, withExtraFields=tc.withExtraFields)
    
    # Compare generated file to reference
    rfn = os.path.join('AutoDS', 'refout', tc.refOutFileName)
    with open(ofn, 'r') as fOut, open(rfn, 'r') as fRef:
        if fOut.read() == fRef.read():
            print('Success : Conform to reference.')
        else:
            print('Error: Generated file differs from reference', rfn)
            fails += 1
            
    print()
    
print('All test cases succeeded !' if fails == 0 else 'Error: {} test case(s) failed.'.format(fails))

# 2. MCDSEngine : Exécution avec de vraies données

In [ ]:
ds = ads.DataSet(dfData=pd.read_excel(os.path.join('AutoDS', 'refin', 'ALAARV-saisie-ttes-cols.xlsx')),
                 decimalFields=['EFFORT', 'DISTANCE', 'NOMBRE'])

eng = ads.MCDSEngine(workDir=os.path.join('AutoDS', 'mcds-out'))

runCode, runDir = eng.run(ds, realRun=True, runPrefix='int',
                          estimKeyFn='UNIFORM', estimAdjustFn='POLY',
                          estimCriterion='AIC', cvInterval=95)
assert runCode == 2, 'Should have run with warnings (run code = 2)'
runDir

# 3. MCDSAnalysis : Analyse avec de vraies données

In [ ]:
_ = implib.reload(ads)

In [ ]:
ds = ads.DataSet(dfData=pd.read_excel(os.path.join('AutoDS', 'refin', 'ALAARV-saisie-ttes-cols.xlsx')),
                 decimalFields=['EFFORT', 'DISTANCE', 'NOMBRE'])

eng = ads.MCDSEngine(workDir=os.path.join('AutoDS', 'mcds-out'))

anlys = ads.MCDSAnalysis(engine=eng, dataSet=ds, namePrefix='mcds',
                         estimKeyFn='HNORMAL', estimAdjustFn='COSINE', estimCriterion='AIC', cvInterval=95)

sRes = anlys.run()

assert sRes[('run output', 'run status', 'Value')] == 2, 'Should have run with warnings (run code = 2)'
sRes[('run output', 'files folder', 'Value')]

In [ ]:
sRes[('run output',)]

## 2. Analyses massives ACDC Papier 2019

In [ ]:
def extraireJeuDonnees(dfTout, espece, passages=['A', 'B'], duree='10mn'):
    
    assert all(p in ['A', 'B'] for p in passages)
    assert duree in ['5mn', '10mn']
    assert espece in dfTout.ESPECE.unique()
    
    # Passages
    dfJeu = dfTout[(dfTout.ESPECE == espece) & (dfTout.PASSAGE.isin(passages))].copy()
    
    # Durée
    if duree == '10mn':
        dfJeu['NOMBRE'] = dfJeu[['PER5MN', 'PER10MN']].sum(axis='columns')
    else:
        dfJeu['NOMBRE'] = dfJeu['PER5MN']
    dfJeu.drop(dfJeu[dfJeu.NOMBRE.isnull()].index, inplace=True)
    assert all(dfJeu.NOMBRE == 1)
        
    # Effort
    dfJeu['EFFORT'] = len(passages)
        
    # Nettoyage
    dfJeu.drop(['PER5MN', 'PER10MN'], axis='columns', inplace=True)
    
    return dfJeu

In [ ]:
def ajouterAbsences(dfJeu, effort, pointsPapier):
    
    assert not dfJeu.empty, 'Erreur : Il n\'y aurait que des absences !'

    zone, surface, espece = dfJeu.iloc[0][['ZONE', 'HA', 'ESPECE']]
    dAbsence = { 'ZONE': zone, 'HA': surface, 'POINT': None, 'ESPECE': espece,
                 'DISTANCE': np.nan, 'EFFORT': effort, 'MALE': None,
                 'NOMBRE': np.nan, 'DATE': pd.NaT, 'OBSERVATEUR': None, 'PASSAGE': None }

    pointsManquants = [p for p in pointsPapier if p not in dfJeu.POINT.unique()]
    for p in pointsManquants:
        dAbsence.update(POINT=p)
        dfJeu = dfJeu.append(dAbsence, ignore_index=True)
    
    dfJeu.sort_values(by=['POINT'], inplace=True)

    return dfJeu, len(pointsManquants)

In [ ]:
# Paramètres généraux.
workDir = os.path.join('AutoDS', 'acdc-auto')
runEngine = True # Pas d'appel à l'exe si False, juste pour les fichiers d'entrée.

In [ ]:
# Tous les points effectués (pour absences).
pointsPapier = \
    list(map(int, """23,39,40,41,42,55,56,57,58,59,60,72,73,74,75,76,88,89,90,91,
                     105,106,109,110,112,113,122,123,125,126,127,128,129,130,141,142,143,144,145,146,
                     147,148,157,158,159,160,161,162,163,164,165,166,174,175,176,177,178,179,180,181,
                     182,183,184,185,192,193,194,195,196,197,198,199,200,201,202,210,211,212,213,214,
                     215,216,218,219,228,229,232,233,245,246,247,250,262,263,265,266,280,281,282,283,
                     284,299,300,301""".split(',')))

# Données brutes saisies par les observateurs, déjà individualisées, que les mâles.
ficDonnees = os.path.join('AutoDS', 'refin', 'ACDC2019-Papyrus-DonneesBrutesPourAutoDS.xlsx')

dfMales = pd.read_excel(ficDonnees, sheet_name='ResultIndivMales')
dfMales.rename(columns={ 'ha': 'HA', 'Distance en m': 'DISTANCE', 'Mâle\xa0?': 'MALE', 'Date': 'DATE',
                         'Période': 'PASSAGE', '0-5mn': 'PER5MN', '5-10 mn': 'PER10MN' }, inplace=True)

assert all(dfMales.MALE.str.lower() == 'oui')

print('Nb mâles   :', len(dfMales))
print('Nb espèces :', len(dfMales.ESPECE.unique()))

# Les espèces et passages à traiter.
dfToDo = pd.read_excel(ficDonnees, sheet_name='AFaire')
toDoCols = ['ESPECE', 'MALES', 'PERIODE']
assert all(col in dfToDo.columns for col in toDoCols)
dfToDo = dfToDo.reindex(toDoCols, axis='columns')
dfToDo.sort_values(by='MALES', ascending=False, inplace=True)

print('Espèces à traiter :', len(dfToDo))

# Les paramètres de toutes les analyses à faire à chaque fois.
dfParams = pd.read_excel(ficDonnees, sheet_name='ParamsAnalyses')
paramCols = ['KeyFn', 'AdjustFn', 'Criterion', 'CVInterval']
assert all(col in paramCols for col in dfParams.columns)
dfParams = dfParams.reindex(paramCols, axis='columns')

print('Variantes d\'analyses :', len(dfParams))

In [ ]:
_ = implib.reload(ads)

In [ ]:
dfToDo[:2]

In [ ]:
# Le moteur
mcds = ads.MCDSEngine(workDir=workDir,
                      distanceUnit='Meter', areaUnit='Hectare',
                      surveyType='Point', distanceType='Radial')

# Les résultats d'analyse
miCustColumns = pd.MultiIndex.from_tuples([('id', 'index', 'Value'),
                                           ('sample', 'species', 'Value'),
                                           ('sample', 'periods', 'Value'),
                                           ('sample', 'duration', 'Value'),
                                           ('variant', 'precision', 'Value')])
resultats = ads.ResultsSet(analysisClass=ads.MCDSAnalysis, customColumns=miCustColumns)

#Pour chaque espèce à traiter
for index, sToDo in dfToDo[:2].iterrows():

    espece, nbIndivs, passage = sToDo
    passages = [p for p in passage]

    # Pour les 2 durées d'inventaire (sur chaque point)
    for duree in ['5mn', '10mn']:

        # Sélection des données
        dfJeu = extraireJeuDonnees(dfMales, espece, passages, duree)
        nMales = len(dfJeu)

        # Ajout des lignes d'absence
        dfJeu, nAbsences = ajouterAbsences(dfJeu, effort=len(passages), pointsPapier=pointsPapier)

        # Pour chaque précision numérique sur la distance (en décroissant)
        for precDist in [None, 1]:
            
            print(espece, passage, duree, ':', nMales, 'mâles,', nAbsences, 'absences')

            # Arrondi à la précision.
            if precDist is not None:
                dfJeu.DISTANCE = dfJeu.DISTANCE.apply(round, ndigits=precDist)
            
            # Voici donc le jeu de données
            jeu = ads.DataSet(dfJeu, decimalFields=['EFFORT', 'DISTANCE', 'NOMBRE'])
            
            # Pour chaque jeu de paramètres d'analyse
            for index, sParams in dfParams.iterrows():

                precision = ('tt' if precDist is None else str(precDist)) + 'dec'
                prfxAnalyse = '{}-{}-{}-{}-{}'.format(espece, duree, passage, precision, index)
                analyse = ads.MCDSAnalysis(engine=mcds, dataSet=jeu, namePrefix=prfxAnalyse,
                                           estimKeyFn=sParams['KeyFn'], estimAdjustFn=sParams['AdjustFn'],
                                           estimCriterion=sParams['Criterion'], cvInterval=sParams['CVInterval'])

                sEntete = pd.Series(data=[index, espece, passage, duree, precision], index=miCustColumns)
                
                sResultat = analyse.run(realRun=runEngine)
                
                resultats.append(sCustom=sEntete, sResult=sResultat)
                                
                #raise StopIteration()
                
            print()

# Sauvegarde des résultats
resultats.dfData.to_excel(os.path.join(workDir, 'ACDC2019-Papyrus-ResultatsAutoAnalyses.xlsx'), index=True)

In [ ]:
resultats.dfData.to_excel(os.path.join(workDir, 'ACDC2019-Papyrus-ResultatsAutoAnalyses.xlsx'), index=True)

In [ ]:
resultats.dfData.head()

# Mise au point décodage sorties de MCDS : fichier de stats

TODO: Add french translation of variables / parameters names and descriptions

## 1. Nom et description des colonnes du tableau de stats

In [ ]:
fileName = 'mcds-stat-row-specs.txt'

fStatRowSpecs = open(fileName, mode='r', encoding='utf8')

In [ ]:
statRowSpecLines = [line.rstrip('\n') for line in fStatRowSpecs.readlines() if not line.startswith('#')]
statRowSpecs =  [(statRowSpecLines[i].strip(), statRowSpecLines[i+1].strip()) \
                 for i in range(0, len(statRowSpecLines)-2, 3)]
dfStatRowSpecs = pd.DataFrame(columns=['Name', 'Description'], data=statRowSpecs).set_index('Name')

dfStatRowSpecs

In [ ]:
dfStatRowSpecs.index

## 2. Numéro et description des modules et statistiques associées

(colonnes Module et Statistic du tableau)

In [ ]:
fileName = 'mcds-stat-mod-specs.txt'

fStatModSpecs = open(fileName, mode='r', encoding='utf8')

In [ ]:
nMaxAdjParams = 10

statModSpecLines = [line.rstrip('\n') for line in fStatModSpecs.readlines() if not line.startswith('#')]
reModSpecNumName = re.compile('(.+) – (.+)')
statModSpecs = list()
moModule = None
for line in statModSpecLines:
    if not line:
        continue
    if moModule is None:
        moModule = reModSpecNumName.match(line.strip())
        continue
    if line == ' ':
        moModule = None
        continue
    moStatistic = reModSpecNumName.match(line.strip())
    modNum, modDesc, statNum, statDescNotes = \
        moModule.group(1), moModule.group(2), moStatistic.group(1), moStatistic.group(2)
    for i in range(len(statDescNotes)-1, -1, -1):
        if not re.match('[\d ,]', statDescNotes[i]):
            statDesc = statDescNotes[:i+1]
            statNotes = statDescNotes[i+1:].replace(' ', '')
            break
    modNum = int(modNum)
    if statNum.startswith('101 '):
        for num in range(nMaxAdjParams): # Assume no more than that ... a bit hacky !
            statModSpecs.append((modNum, modDesc, 101+num, # Make statDesc unique for later indexing
                                 statDesc.replace('each', 'A({})'.format(num+1)), statNotes))
    else:
        statNum = int(statNum)
        if modNum == 2 and statNum == 3: # Actually, there are 0 or 3 of these ...
            for num in range(3):
                statModSpecs.append((modNum, modDesc, num+201,
                                     # Change statNum & Make statDesc unique for later indexing
                                     statDesc+' (distance set {})'.format(num+1), statNotes))
        else:
            statModSpecs.append((modNum, modDesc, statNum, statDesc, statNotes))
dfStatModSpecs = pd.DataFrame(columns=['modNum', 'modDesc', 'statNum', 'statDesc', 'statNotes'],
                              data=statModSpecs).set_index(['modNum', 'statNum'])

dfStatModSpecs

In [ ]:
# Modules
dfStatModSpecs.modDesc.unique()

## 3. Notes sur les statistiques des modules

(infos supplémentaire indiquant comment utiliser ou pas les 5 dernières colonnes Value, Cv, Lcl, Ucl, Df)

In [ ]:
fileName = 'mcds-stat-mod-notes.txt'

fStatModNotes = open(fileName, mode='r', encoding='utf8')

In [ ]:
statModNoteLines = [line.rstrip('\n') for line in fStatModNotes.readlines() if not line.startswith('#')]
statModNotes =  [(int(line[:2]), line[2:].strip()) for line in statModNoteLines if line]

dfStatModNotes = pd.DataFrame(data=statModNotes, columns=['Note', 'Text']).set_index('Note')

dfStatModNotes

## 4. Lecture du tableau

In [ ]:
eng = mcds

In [ ]:
eng.statsFileName

In [ ]:
dfStatRows = pd.read_csv(eng.statsFileName, sep=' +', engine='python', names=dfStatRowSpecs.index)
dfStatRows

## 5. Décodage du tableau

Attention: On suppose 1 seule strate '0' (Stratum), 1 seul échantillon '0' (Sample) et 1 seul estimateur '1' (Estimator).

### a. Suppression des colonnes Stratum, Sample et Estimator

(puisqu'on se limite ici aux cas où il n'y a qu'1 de chaque)

In [ ]:
dfStatRows.drop(columns=['Stratum', 'Sample', 'Estimator'], inplace=True)
dfStatRows

### b. Nettoyage des données sans objets

(selon les notes descriptives des statistiques)

In [ ]:
# Empilage des "chiffres" (Figures) Value, Cv, Lcl, Ucl, Df pour chaque statistique / module
dfStats = dfStatRows.set_index(['Module', 'Statistic'], append=True).stack() \
                    .reset_index().rename(columns={'level_0': 'id', 'level_3': 'Figure', 0: 'Value'})
dfStats.head(10)

In [ ]:
# 4. Fix multiple Module=2 & Statistic=3 rows (before joining with self.DfStatModSpecs)
newStatNum = 200
for lbl, sRow in dfStats[(dfStats.Module == 2) & (dfStats.Statistic == 3)].iterrows():
    if dfStats.loc[lbl, 'Figure'] == 'Value':
        newStatNum += 1
    dfStats.loc[lbl, 'Statistic'] = newStatNum
dfStats[(dfStats.Module == 2)]

In [ ]:
# Ajout des colonnes de description/nommage des modules et statistiques
dfStats = dfStats.join(dfStatModSpecs, on=['Module', 'Statistic'])
dfStats.tail(10)

In [ ]:
#dfStats[(dfStats.Module == 2) & (dfStats.Statistic > 200)]

In [ ]:
# Vérification que les chiffres sans objet le sont vraiment (tous à 0.0 ?)
# Attention: Il doit y avoir un bug dans MCDS avec Module 2 / Statistic 10x : certains Cv ne sont pas nuls ...
sKeepOnlyValueFig = ~dfStats.statNotes.str.contains('1')
sFigs2Drop = (dfStats.Figure != 'Value') & sKeepOnlyValueFig
assert ~dfStats[sFigs2Drop & ((dfStats.Module != 2) | (dfStats.Statistic < 100))].Value.any(), \
       'Attention: Des chiffres supposés "sans objet" on des valeurs non nulles !'

In [ ]:
# 2nde vérif. visuelle
dfStats[sFigs2Drop & dfStats.Value != 0].sort_values(by='Value', ascending=False)

In [ ]:
# Suppression des lignes / chiffres sans objet.
dfStats.drop(dfStats[sFigs2Drop].index, inplace=True)
dfStats

In [ ]:
dfStats.head()

In [ ]:
dfStats = dfStats.reindex(columns=['modDesc', 'statDesc', 'Figure', 'Value'])
dfStats.set_index(['modDesc', 'statDesc', 'Figure'], inplace=True)
dfStats

In [ ]:
dfStats.T.iloc[0]

# Bac à sable

## Appending series to Series ... index order

In [ ]:
s = pd.Series(index=pd.MultiIndex.from_tuples([('B', 'b'), ('B', 'a'), ('A', 'c')]), data=[1, 2, 3], name=0)
s

In [ ]:
s.append(pd.Series(index=[('A', 'b'), ('A', 'a'), ('B', 'c')], data=[1, 2, 3], name=0))

## Appending series to DataFrame ... columns order

### a. Append

In [ ]:
df = pd.DataFrame()

In [ ]:
s = pd.Series(index=pd.MultiIndex.from_tuples([('B', 'b'), ('B', 'a'), ('A', 'c')]), data=[1, 2, 3], name=0)
#df = df.append(s, ignore_index=False) # => df.columns pas MultiIndex !
df = df.append([s], ignore_index=False)
df

In [ ]:
s = pd.Series(index=[('A', 'c'), ('B', 'b'), ('B', 'a')], data=[4, 5, 6], name=1)  # Mêmes colonnes : append ne retrie pas
#s = pd.Series(index=[('A', 'a'), ('A', 'b'), ('B', 'c')], data=[4, 5, 6], name=1)  # Nouvelle colonne : append retrie
df = df.append([s], ignore_index=True)
df

In [ ]:
s = pd.Series(index=[('A', 'a'), ('B', 'c')], data=[7, 8])
df = df.append(s, ignore_index=True)
df

In [ ]:
s = pd.Series(index=[], data=[])
df = df.append([s], ignore_index=True)
df

In [ ]:
s = pd.Series(index=[('C', 'd')], data=[9])
df = df.append([s], ignore_index=True)
df

In [ ]:
s = pd.Series(index=[('d',)], data=[10])
df = df.append(s, ignore_index=True)
df

In [ ]:
df

### b. Concat

In [ ]:
df = pd.DataFrame()

In [ ]:
s = pd.Series(index=pd.MultiIndex.from_tuples([('B', 'b'), ('B', 'a'), ('A', 'c')]), data=[1, 2, 3], name=0)
df = pd.concat([df, s], axis='columns')
df

In [ ]:
s = pd.Series(index=[('B', 'b'), ('B', 'a'), ('A', 'c')], data=[4, 5, 6], name=1) # Mêmes colonnes : concat ne retrie pas
#s = pd.Series(index=[('A', 'a'), ('A', 'b'), ('B', 'c')], data=[4, 5, 6], name=1) # Nouvelle colonne : concat retrie
df = pd.concat([df, s], axis='columns')
df

### c. Restore desired columns order.

In [ ]:
i = pd.MultiIndex.from_tuples([('C', 'd'), ('A', 'c'), ('A', 'a'), ('B', 'c'), ('B', 'b'), ('B', 'a')])
i

In [ ]:
df

In [ ]:
df.reindex(i, axis='columns')